Code qui marche mais qui n'a pas de continuité aux bords

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Script pour simuler la dynamique d'une population de cellules avec signalisation chimique.
Auteur: Alexander Golden

Ce script simule l'évolution d'un champ de signal chimique (par exemple, AMPc) diffusif couplé à la dynamique cellulaire.
Les résultats (images du champ de signal et séries temporelles) sont sauvegardés dans le dossier 'output_files'.
"""

### Import des bibliothèques ###
from __future__ import division  # Pour garantir la division réelle (utile en Python 2)
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Utilisation d'un backend non interactif pour sauvegarder les figures
import matplotlib.pyplot as plt
import math
import os
import sys
import scipy.stats as sc
import csv
import pandas as pd
import torch
import time
import scipy as sp
import scipy.sparse as sparse
from numba import njit, float64
from matplotlib.colors import LogNorm
import itertools as it
import random as rand
import pickle as pkl
import argparse

### Paramètres par défaut ###
dx = 0.5              # Pas spatial (distance entre deux points de la grille)
ll = 50               # Taille de la boîte (domaine de simulation en x et y)
agent_dim = 2 * dx    # Dimension caractéristique d'un agent

# Paramètres cellulaires par défaut
cell_params_default = {
    'c0': 1,            # Terme constant dans la dynamique cellulaire
    'a': 0.05,          # Coefficient d'activation (sensibilité au signal)
    'gamma': 0.5,       # Coefficient d'inhibition (régulation négative)
    'Kd': 10**(-5),     # Constante de dissociation (sensibilité)
    'sigma': 0.15,      # Amplitude du bruit dans la dynamique
    'epsilon': 0.2,     # Échelle temporelle pour la variable R
    'cStim': 100,       # Niveau de stimulation externe
    'aPDE': 10,         # Coefficient de dégradation du signal
    'rho': 1,           # Taux de production du signal
    'D': 1000,          # Coefficient additionnel lié à la production par les agents
    'a0': 1,            # Terme constant de production du signal
    'af': 0             # Paramètre additionnel (souvent inutilisé)
}

# Paramètres de la grille par défaut
grid_params_default = {
    'dx': dx,              # Pas spatial
    'D_sig': 5,            # Coefficient de diffusion du signal
    'box_size_x': ll,      # Taille du domaine en x
    'box_size_y': ll,      # Taille du domaine en y
    'agent_dim': agent_dim,  # Dimension caractéristique d'un agent
    'num_agents': 15000    # Nombre total d'agents (cellules)
}

### Fonctions de signalisation et utilitaires ###

@njit(float64[:,:](float64, float64[:,:], float64, float64, float64, float64, float64, float64[:,:], float64[:,:]))
def addcD(dt, lap, Dsig, J, rho, a0, D, signal, A_grid_hsided):
    """
    Calcule l'incrément du signal chimique (dC) sur un pas de temps dt en combinant diffusion et réaction.
    
    dC = dt*(Dsig*lap - J*signal + rho*a0) + dt*(rho*D*A_grid_hsided)
    
    Parameters:
      dt (float): Pas de temps.
      lap (ndarray): Laplacien du signal.
      Dsig (float): Coefficient de diffusion.
      J (float): Coefficient de réaction (aPDE).
      rho (float): Taux de production.
      a0 (float): Terme constant de production.
      D (float): Coefficient additionnel pour la production par agents.
      signal (ndarray): Champ de signal actuel.
      A_grid_hsided (ndarray): Grille d'agents après application d'une fonction seuil.
    
    Returns:
      ndarray: Incrément du signal.
    """
    return dt * (Dsig * lap - J * signal + rho * a0) + dt * (rho * D * A_grid_hsided)

def accumulate_arr(coord, arr, shape):
    """
    Accumule les valeurs d'un tableau d'agents dans une grille 2D en fonction de leurs coordonnées.
    
    Parameters:
      coord (ndarray): Coordonnées des agents, de forme (2, nombre_d'agents).
      arr (ndarray): Valeurs associées à chaque agent.
      shape (tuple): Dimensions (lignes, colonnes) de la grille.
    
    Returns:
      ndarray: Grille 2D avec les valeurs accumulées.
    """
    lidx = np.ravel_multi_index(coord, shape)
    return np.bincount(lidx, arr, minlength=shape[0]*shape[1]).reshape(shape[0], shape[1])

def scale(A, B, k):
    """
    Remplit la matrice A avec la matrice B répétée en blocs de taille k x k.
    
    Parameters:
      A (ndarray): Matrice de destination.
      B (ndarray): Matrice source.
      k (int): Facteur d'échelle (taille du bloc).
    
    Returns:
      ndarray: Matrice A modifiée.
    """
    Y, X = A.shape
    for y in range(0, k):
        for x in range(0, k):
            A[y:Y:k, x:X:k] = B
    return A

def scale1D(A, B, k):
    """
    Version 1D de la fonction scale.
    
    Parameters:
      A (ndarray): Vecteur de destination.
      B (ndarray): Vecteur source.
      k (int): Facteur d'échelle.
    
    Returns:
      ndarray: Vecteur A modifié.
    """
    Y = A.shape[0]
    for y in range(0, k):
        A[y:Y:k] = B
    return A

def scale_down_mat_sp(current_shape, scale):
    """
    Génère une matrice creuse permettant de réduire une matrice d'origine par un facteur 'scale'
    en effectuant la moyenne sur des blocs.
    
    Parameters:
      current_shape (tuple): Dimensions de la matrice d'origine.
      scale (int): Facteur de réduction (doit diviser exactement les dimensions).
    
    Returns:
      csr_matrix: Matrice de downscaling.
    """
    assert current_shape[0] // scale == current_shape[0] / scale, "Shape not divisible by scale"
    assert current_shape[1] // scale == current_shape[1] / scale, "Shape not divisible by scale"
    out_shape = (current_shape[0] // scale, current_shape[1] // scale)
    line = np.array(([1/scale**2]*scale + [0]*(current_shape[0]-scale))*scale + [0]*(current_shape[0]*(current_shape[1]-scale)))
    out_mat = sp.sparse.bsr_matrix(line)
    for i in range(out_shape[0]-1):
        newline = np.roll(line, scale*(i+1))
        out_mat = sp.sparse.vstack([out_mat, newline])
    block = out_mat.todense()
    for i in range(out_shape[1]-1):
        newblock = sp.sparse.bsr_matrix(np.roll(block, scale*current_shape[0]*(i+1), axis=1))
        out_mat = sp.sparse.vstack([out_mat, newblock])
    return sparse.csr_matrix(out_mat)

def calc_square_laplacian_noflux_matrix(size):
    """
    Calcule la matrice laplacienne 2D avec conditions aux limites "no flux" pour une grille donnée.
    
    Parameters:
      size (tuple): Dimensions (lignes, colonnes) de la grille.
    
    Returns:
      csr_matrix: Matrice laplacienne compressée.
    """
    xDiff = np.zeros((size[1]+1, size[1]))
    ix, jx = np.indices(xDiff.shape)
    xDiff[ix == jx] = 1
    xDiff[ix == jx+1] = -1
    yDiff = np.zeros((size[0]+1, size[0]))
    iy, jy = np.indices(yDiff.shape)
    yDiff[iy == jy] = 1
    yDiff[iy == jy+1] = -1
    Ax = sparse.dia_matrix(-np.matmul(np.transpose(xDiff), xDiff))
    Ay = sparse.dia_matrix(-np.matmul(np.transpose(yDiff), yDiff))
    lap = sparse.kron(Ay, sparse.eye(size[1]), format='csr') + sparse.kron(sparse.eye(size[0]), Ax, format='csr')
    lap += sparse.diags([2] + [1]*(size[1]-2) + [2] + ([1] + [0]*(size[1]-2) + [1])*(size[0]-2) + [2] + [1]*(size[1]-2) + [2])
    lap = sparse.bsr_matrix(lap)
    return lap

def smooth(y, box_pts):
    """
    Lisse une série de données par convolution avec une fenêtre rectangulaire.
    
    Parameters:
      y (ndarray): Série de données.
      box_pts (int): Taille de la fenêtre.
    
    Returns:
      ndarray: Série lissée.
    """
    box = np.ones(box_pts) / box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

### Classe de simulation de la population de cellules ###
class dictyPop:
    def __init__(self, T, save_every, g_params=grid_params_default, c_params=cell_params_default, noise=True, progress_report=True, save_data=True):
        """
        Initialise la simulation de la population de cellules avec signalisation chimique.
        
        Parameters:
          T (float): Temps total de simulation.
          save_every (float): Intervalle de sauvegarde des données.
          g_params (dict): Paramètres de la grille.
          c_params (dict): Paramètres cellulaires.
          noise (bool): Active ou désactive le bruit dans la dynamique.
          progress_report (bool): Affiche la progression de la simulation.
          save_data (bool): Active la sauvegarde des données (pour séries temporelles).
        """
        self.g_params = g_params
        self.c_params = c_params
        self.T = T
        self.ts = 0  # Temps courant initialisé à 0
        self.dt = self.g_params['dx']**2 / (8 * self.g_params['D_sig'])  # Calcul du pas de temps basé sur la diffusion
        self.Tsteps = int(self.T / self.dt)
        # Dimensions de la grille du signal
        self.signal_size = (int(self.g_params['box_size_x'] / self.g_params['dx']),
                            int(self.g_params['box_size_y'] / self.g_params['dx']))
        self.fluxes = np.zeros(4)  # Initialisation des flux aux bords
        self.p_r = progress_report
        self.save_every = save_every
        self.save_data = save_data
        # Ratio pour downscaling (liaison entre la grille des agents et celle du signal)
        self.agent_ratio = int(self.g_params['agent_dim'] / self.g_params['dx'])
        self.agent_size = (max(1, int(self.g_params['box_size_x'] / self.g_params['agent_dim'])),
                           max(1, int(self.g_params['box_size_y'] / self.g_params['agent_dim'])))
        self.signal = np.zeros(self.signal_size)  # Champ de signal initialisé à zéro
        self.noise_flag = noise
        # Initialisation des états cellulaires (A et R) pour chaque agent, tirés d'une distribution normale
        cell_state = np.random.normal(loc=0.0, scale=2, size=(2, g_params['num_agents']))
        self.A = cell_state[0, :]
        self.R = cell_state[1, :]
        self.D = float(self.c_params['D'])
        self.dsignal = np.zeros(self.signal.shape)
        self.dA = np.zeros(self.A.shape)
        self.dR = np.zeros(self.R.shape)
        # Matrice d'interpolation pour downscaling du signal vers la grille des agents
        self.interp_mat = scale_down_mat_sp(self.signal_size, self.agent_ratio)
        # Grille pour accumuler les valeurs de A des agents
        self.A_grid = np.zeros((self.signal_size[0] // self.agent_ratio, self.signal_size[1] // self.agent_ratio))
        # Grille "agrandie" pour interpolation (utilisée dans la production du signal)
        self.A_grid_big = np.zeros(self.signal_size)
        # Coordonnées des agents dans la grille réduite
        self.coord = np.zeros((2, self.g_params['num_agents']), dtype='int')
        for agent in range(self.g_params['num_agents']):
            self.coord[:, agent] = np.array([
                int(rand.random() * ll / max(agent_dim, dx)),
                int(rand.random() * ll / max(agent_dim, dx))
            ])
        # Construction de la matrice laplacienne avec conditions "no flux"
        if min(self.signal_size) > 1:
            self.lap_mat = calc_square_laplacian_noflux_matrix(self.signal_size)
        else:
            self.lap_mat = sparse.diags([np.ones(max(self.signal_size)-1),
                                         -2*np.ones(max(self.signal_size)),
                                         np.ones(max(self.signal_size)-1)],
                                        [-1, 0, 1], format='csr')
            self.lap_mat += sparse.diags(np.array([1] + [0]*(max(self.signal_size)-2) + [1]), format='csr')
        # Sauvegarde des données de la simulation (pour séries temporelles)
        if self.save_data is True:
            num_save = int(self.T // self.save_every) + 1  # Conversion en entier
            self.A_saved = np.zeros((self.signal_size[0] // self.agent_ratio,
                                     self.signal_size[1] // self.agent_ratio,
                                     num_save))
            self.R_saved = np.zeros((self.signal_size[0] // self.agent_ratio,
                                     self.signal_size[1] // self.agent_ratio,
                                     num_save))
            self.S_saved = np.zeros((self.signal_size[0],
                                     self.signal_size[1],
                                     num_save))
            self.A_saved[:, :, 0] = self.getAGrid()
            self.R_saved[:, :, 0] = self.getRGrid()
            self.S_saved[:, :, 0] = self.signal

    def getAGrid(self):
        """
        Retourne la grille 2D des valeurs de A accumulées selon les coordonnées des agents.
        
        Returns:
          ndarray: Grille des valeurs de A.
        """
        return accumulate_arr(self.coord, self.A, self.A_grid.shape)

    def getRGrid(self):
        """
        Retourne la grille 2D des valeurs de R accumulées selon les coordonnées des agents.
        
        Returns:
          ndarray: Grille des valeurs de R.
        """
        return accumulate_arr(self.coord, self.R, self.A_grid.shape)

    def setSignal(self, signal):
        """
        Définit le champ de signal.
        
        Parameters:
          signal (ndarray): Nouvelle matrice du signal.
        """
        assert (self.signal.shape is signal.shape) and (self.signal.dtype is signal.dtype), "Input signal incorrect shape or type"
        self.signal = np.array(signal)

    def setCellState(self, state):
        """
        Définit l'état cellulaire (A et R) pour tous les agents.
        
        Parameters:
          state (ndarray): Tableau de forme (2, nombre_d'agents) avec les valeurs de A et R.
        """
        assert (self.A.shape == state[:, 0].shape) and (self.A.dtype == state.dtype), "Input cell state incorrect shape or type"
        self.A = np.array(state[:, 0])
        self.R = np.array(state[:, 1])

    def setFluxes(self, fluxes):
        """
        Définit les flux appliqués aux bords.
        
        Parameters:
          fluxes (ndarray): Tableau de forme (4,) contenant les flux pour chaque bord.
        """
        assert (self.fluxes.shape is fluxes.shape) and (self.fluxes.dtype is fluxes.dtype), "Input fluxes incorrect shape or type"
        self.fluxes = np.array(fluxes)

    def getdA(self):
        """
        Calcule l'incrément de A pour chaque agent en combinant la dynamique interne (FitzHugh–Nagumo)
        et la contribution liée au signal local.
        
        Returns:
          ndarray: Incrément de A.
        """
        return ((self.A - (self.A*self.A*self.A)/3 - self.R)) * self.dt \
               + ((self.c_params['a'] * np.log1p(
                   np.reshape(self.interp_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
                              self.agent_size)[self.coord[0, :], self.coord[1, :]] / self.c_params['Kd']
               )))*self.dt

    def getdR(self):
        """
        Calcule l'incrément de R pour chaque agent selon la dynamique cellulaire.
        
        Returns:
          ndarray: Incrément de R.
        """
        return (((self.A - self.c_params['gamma']*self.R) + self.c_params['c0'])
                * self.c_params['epsilon'])*self.dt

    def getdC(self):
        """
        Calcule l'incrément de la concentration du signal en combinant diffusion et flux aux bords.
        
        Returns:
          ndarray: Incrément du signal.
        """
        laplacian = 1/(self.g_params['dx']**2) * np.reshape(
            self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
            (self.signal_size[0], self.signal_size[1])
        )
        # Application manuelle des flux aux bords
        self.signal[0, 1:-1] -= self.fluxes[0] / self.g_params['dx']
        self.signal[-1, 1:-1] -= self.fluxes[1] / self.g_params['dx']
        self.signal[1:-1, 0] -= self.fluxes[2] / self.g_params['dx']
        self.signal[1:-1, -1] -= self.fluxes[3] / self.g_params['dx']
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        return self.dt*(self.g_params['D_sig']*laplacian - self.c_params['aPDE']*self.signal + self.c_params['rho']*self.c_params['a0']) \
               + self.dt*(self.c_params['rho']*self.c_params['D']*np.heaviside(self.A_grid_big, 0.5))
    
    def getLapC(self):
        """
        Calcule le laplacien du champ de signal.
        
        Returns:
          ndarray: Laplacien sous forme d'une matrice 2D.
        """
        return 1/(self.g_params['dx']**2) * np.reshape(
            self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0]*self.signal_size[1])),
            (self.signal_size[0], self.signal_size[1])
        )
    
    def update(self):
        """
        Met à jour la simulation pour un pas de temps donné.
        
        Returns:
          ndarray: Nouveau champ de signal.
        """
        self.ts += self.dt  # Incrémente le temps courant
        self.dA = self.getdA()  # Calcul de l'incrément de A
        if self.noise_flag:
            # Ajout de bruit gaussien à A
            self.dA += self.c_params['sigma'] * np.random.normal(loc=0.0, scale=np.sqrt(self.dt), size=self.A.shape)
        self.dR = self.getdR()  # Calcul de l'incrément de R
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        self.dsignal = addcD(self.dt, self.getLapC(), self.g_params['D_sig'], 
                             self.c_params['aPDE'], self.c_params['rho'], 
                             self.c_params['a0'], self.c_params['D'],
                             self.signal, np.heaviside(self.A_grid_big, 0.5))
        # Mise à jour des états cellulaires et du signal
        self.A += self.dA
        self.R += self.dR
        self.signal += self.dsignal
        # Application des flux aux bords
        self.signal[0, 1:-1] -= self.fluxes[0] / self.g_params['dx']
        self.signal[-1, 1:-1] -= self.fluxes[1] / self.g_params['dx']
        self.signal[1:-1, 0] -= self.fluxes[2] / self.g_params['dx']
        self.signal[1:-1, -1] -= self.fluxes[3] / self.g_params['dx']
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3]) / (2 * self.g_params['dx'])
        return self.signal

### Fonction pour générer des séries temporelles ###
def timeseries_a(a, c0, t0, tf, freq, save=100, size=10):
    """
    Génère une série temporelle de l'intensité du signal en sauvegardant des images et en accumulant
    des moyennes locales.
    
    Parameters:
      a (float): Coefficient d'activation modifié.
      c0 (float): Terme constant modifié pour la dynamique cellulaire.
      t0 (int): Temps initial (pas de départ de la simulation).
      tf (int): Temps final (nombre total de pas).
      freq (float): Fréquence d'enregistrement (non utilisé ici).
      save (int): Intervalle de sauvegarde en nombre de pas.
      size (int): Taille du bloc pour le calcul des moyennes locales.
    
    Returns:
      ndarray: Série temporelle des intensités de signal normalisées.
    """
    path = 'output_files/'
    os.makedirs(path, exist_ok=True)  # Crée le dossier de sortie s'il n'existe pas
    N = int(15000)  # Taille de la population
    
    # Mise à jour des paramètres cellulaires
    cell_params = cell_params_default.copy()
    cell_params['c0'] = c0
    cell_params['a'] = a
    
    # Paramètres de la grille
    grid_params = {
        'dx': dx,
        'D_sig': 5,
        'box_size_x': ll,
        'box_size_y': ll,
        'agent_dim': agent_dim,
        'num_agents': N
    }
    
    noise = True
    # Initialisation de la population avec T=1 et save_every=1 pour cette fonction
    pop = dictyPop(1, 1, c_params=cell_params, g_params=grid_params, noise=noise)
    # Réinitialisation des états A et R à zéro
    pop.A = np.zeros(pop.A.shape)
    pop.R = np.zeros(pop.R.shape)
    S = pop.update()
    
    num_regions = int((len(S) / size)**2)
    TimeSeries = np.full((num_regions, 1), np.nan)
    
    # Boucle de simulation sur l'intervalle [t0, tf)
    for i in range(t0, tf):
        S = pop.update()
        NewCol = []
        # Sauvegarde d'une image tous les 100 pas
        if i % 100 == 0:
            plt.matshow(S)
            plt.title("t=" + str(i))
            plt.savefig(path + str(i) + '.png')
            plt.close()
        # Calcul des moyennes locales et mise à jour de la série temporelle tous les 'save' pas
        if i % save == 0:
            for i in range(0, len(S), size):
                for j in range(0, len(S), size):
                    if i + size <= len(S) and j + size <= len(S):
                        mean_val = np.mean(S[i:i+size, j:j+size])
                        if np.mean(S) > 0:
                            NewCol.append(mean_val / np.mean(S))
                        else:
                            NewCol.append(mean_val)
            if len(NewCol) == num_regions:
                NCol = pd.DataFrame({'col': NewCol})
                TimeSeries = np.column_stack((TimeSeries, NCol))
    
    return TimeSeries

### Exécution du script et production des séries temporelles ###
# Définir les variables manquantes
ll = 50  # Taille de la zone (domaine de simulation)
agent_dim = 2 * 0.5  # 0.5 est la valeur de dx
size = 50  # Taille pour calculer les moyennes locales

# Paramètres de la simulation
t0 = 0         # Pas de départ
tf = 100000    # Nombre total de pas
freq = 0.1     # Fréquence d'enregistrement (non utilisé ici)
a = 0.04       # Coefficient d'activation modifié
c0 = 0.5         # Terme constant modifié
# Lancer la simulation pour générer les séries temporelles
resultats = timeseries_a(a, c0, t0, tf, freq, 100, 10)

# Sauvegarder les résultats dans un fichier CSV
df = pd.DataFrame(resultats)
df.to_csv("timeseries_signal.csv", index=False)

# Visualiser les résultats avec un tracé
plt.plot(resultats)
plt.xlabel("Temps")
plt.ylabel("Intensité du signal")
plt.title("Séries temporelles de l'intensité du signal")
plt.show()

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Script pour simuler la dynamique d'une population de cellules avec signalisation chimique.
Auteur: Alexander Golden
"""

### Import des bibliothèques ###
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Utilisation du backend non interactif pour matplotlib
import matplotlib.pyplot as plt
import math
import os
import sys
import scipy.stats as sc
import csv
import pandas as pd
import torch
import time
import scipy as sp
import scipy.sparse as sparse
from numba import njit, float64
from matplotlib.colors import LogNorm
import itertools as it
import random as rand
import pickle as pkl
import argparse

### Fonctions de signalisation ###

@njit(float64[:,:](float64, float64[:,:], float64, float64, float64, float64, float64, float64[:,:], float64[:,:]))
def addcD(dt, lap, Dsig, J, rho, a0, D, signal, A_grid_hsided):
    """
    Fonction pour mettre à jour la concentration du signal chimique.
    """
    return dt * (Dsig * lap - J * signal + rho * a0) + dt * (rho * D * A_grid_hsided)

def accumulate_arr(coord, arr, shape):
    """
    Accumule les valeurs dans un tableau 2D en fonction des coordonnées.
    """
    lidx = np.ravel_multi_index(coord, shape)
    return np.bincount(lidx, arr, minlength=shape[0]*shape[1]).reshape(shape[0], shape[1])

def scale(A, B, k):
    """
    Remplit la matrice A avec la matrice B mise à l'échelle par k.
    """
    Y = A.shape[0]
    X = A.shape[1]
    for y in range(0, k):
        for x in range(0, k):
            A[y:Y:k, x:X:k] = B
    return A

def scale1D(A, B, k):
    """
    Remplit la matrice A avec la matrice B mise à l'échelle par k (version 1D).
    """
    Y = A.shape[0]
    for y in range(0, k):
        A[y:Y:k] = B
    return A

def scale_down_mat_sp(current_shape, scale):
    """
    Génère une matrice pour réduire la taille d'une matrice par un facteur donné.
    """
    assert current_shape[0] // scale == current_shape[0] / scale, "Shape not divisible by scale"
    assert current_shape[1] // scale == current_shape[1] / scale, "Shape not divisible by scale"
    out_shape = (current_shape[0] // scale, current_shape[1] // scale)
    line = np.array(([1/scale**2]*scale + [0]*(current_shape[0]-scale))*scale + [0]*(current_shape[0]*(current_shape[1]-scale)))
    out_mat = sp.sparse.bsr_matrix(line)
    for i in range(out_shape[0]-1):
        newline = np.roll(line, scale*(i+1))
        out_mat = sp.sparse.vstack([out_mat, newline])
    block = out_mat.todense()
    for i in range(out_shape[1]-1):
        newblock = sp.sparse.bsr_matrix(np.roll(block, scale*current_shape[0]*(i+1), 1))
        out_mat = sp.sparse.vstack([out_mat, newblock])
    return sparse.csr_matrix(out_mat)

def calc_square_laplacian_noflux_matrix(size):
    """
    Calcule la matrice Laplacienne pour des conditions aux limites de flux nul.
    """
    xDiff = np.zeros((size[1]+1, size[1]))
    ix, jx = np.indices(xDiff.shape)
    xDiff[ix==jx] = 1
    xDiff[ix==jx+1] = -1
    yDiff = np.zeros((size[0]+1, size[0]))
    iy, jy = np.indices(yDiff.shape)
    yDiff[iy==jy] = 1
    yDiff[iy==jy+1] = -1
    Ax = sparse.dia_matrix(-np.matmul(np.transpose(xDiff), xDiff))
    Ay = sparse.dia_matrix(-np.matmul(np.transpose(yDiff), yDiff))
    lap = sparse.kron(Ay, sparse.eye(size[1]), format='csr') + sparse.kron(sparse.eye(size[0]), Ax, format='csr')
    lap += sparse.diags([2] + [1]*(size[1]-2) + [2] + ([1] + [0]*(size[1]-2) + [1])*(size[0]-2) + [2] + [1]*(size[1]-2) + [2])
    lap = sparse.bsr_matrix(lap)
    return lap

def smooth(y, box_pts):
    """
    Lisse un signal en utilisant une convolution avec une fenêtre rectangulaire.
    """
    box = np.ones(box_pts) / box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

### Population de cellules signalisantes ###

class dictyPop:
    def __init__(self, T, save_every, g_params=grid_params_default, c_params=cell_params_default, noise=True, progress_report=True, save_data=True):
        """
        Initialise la population de cellules avec les paramètres donnés.
        """
        self.g_params = g_params
        self.c_params = c_params
        self.T = T
        self.ts = 0
        self.dt = self.g_params['dx']**2 / (8 * self.g_params['D_sig'])  # Pas de temps basé sur la dimension spatiale
        self.Tsteps = int(self.T / self.dt)
        self.signal_size = (int(self.g_params['box_size_x'] / self.g_params['dx']), int(self.g_params['box_size_y'] / self.g_params['dx']))
        self.fluxes = np.zeros(4)
        self.p_r = progress_report
        self.save_every = save_every
        self.save_data = save_data
        self.agent_ratio = int(self.g_params['agent_dim'] / self.g_params['dx'])
        self.agent_size = (max(1, int(self.g_params['box_size_x'] / self.g_params['agent_dim'])), max(1, int(self.g_params['box_size_y'] / self.g_params['agent_dim'])))
        self.signal = np.zeros(self.signal_size)
        self.noise_flag = noise
        cell_state = np.random.normal(loc=0.0, scale=2, size=(2, g_params['num_agents']))
        self.A = cell_state[0, :]
        self.R = cell_state[1, :]
        self.D = float(self.c_params['D'])
        self.dsignal = np.zeros(self.signal.shape)
        self.dA = np.zeros(self.A.shape)
        self.dR = np.zeros(self.R.shape)
        self.interp_mat = scale_down_mat_sp(self.signal_size, self.agent_ratio)
        self.A_grid = np.zeros((self.signal_size[0] // self.agent_ratio, self.signal_size[1] // self.agent_ratio))
        self.A_grid_big = np.zeros(self.signal.shape)
        self.coord = np.zeros((2, self.g_params['num_agents']), dtype='int')
        for agent in range(self.g_params['num_agents']):
            self.coord[:, agent] = np.array([int(rand.random() * ll / max(agent_dim, dx)), int(rand.random() * ll / max(agent_dim, dx))])
        if min(self.signal_size) > 1:
            self.lap_mat = calc_square_laplacian_noflux_matrix(self.signal_size)
        else:
            self.lap_mat = sparse.diags([np.ones(max(self.signal_size)-1), -2*np.ones(max(self.signal_size)), np.ones(max(self.signal_size)-1)], [-1, 0, 1], format='csr')
            self.lap_mat += sparse.diags(np.array([1] + [0]*(max(self.signal_size)-2) + [1]), format='csr')
        if self.save_data is True:
            self.A_saved = np.zeros((self.signal_size[0] // self.agent_ratio, self.signal_size[1] // self.agent_ratio, (self.T // save_every) + 1))
            self.R_saved = np.zeros((self.signal_size[0] // self.agent_ratio, self.signal_size[1] // self.agent_ratio, (self.T // save_every) + 1))
            self.S_saved = np.zeros((self.signal_size[0], self.signal_size[1], (self.T // save_every) + 1))
            self.A_saved[:, :, 0] = self.getAGrid()
            self.R_saved[:, :, 0] = self.getRGrid()
            self.S_saved[:, :, 0] = self.signal

    def getAGrid(self):
        """
        Retourne la grille des valeurs de A.
        """
        return accumulate_arr(self.coord, self.A, self.A_grid.shape)

    def getRGrid(self):
        """
        Retourne la grille des valeurs de R.
        """
        return accumulate_arr(self.coord, self.R, self.A_grid.shape)

    def setSignal(self, signal):
        """
        Définit le signal chimique.
        """
        assert (self.signal.shape == signal.shape) and (self.signal.dtype == signal.dtype), "Input signal incorrect shape or type"
        self.signal = np.array(signal)

    def setCellState(self, state):
        """
        Définit l'état des cellules.
        """
        assert (self.A.shape == state[:, 0].shape) and (self.A.dtype == state.dtype), "Input cell state incorrect shape or type"
        self.A = np.array(state[:, 0])
        self.R = np.array(state[:, 1])

    def setFluxes(self, fluxes):
        """
        Définit les flux aux bords.
        """
        assert (self.fluxes.shape == fluxes.shape) and (self.fluxes.dtype == fluxes.dtype), "Input fluxes incorrect shape or type"
        self.fluxes = np.array(fluxes)

    def getdA(self):
        """
        Calcule la dérivée de A.
        """
        return ((self.A - (self.A * self.A * self.A) / 3 - self.R)) * self.dt + ((self.c_params['a'] * np.log1p(np.reshape(self.interp_mat.dot(np.reshape(self.signal, self.signal_size[0] * self.signal_size[1])), self.agent_size)[self.coord[0, :], self.coord[1, :]] / self.c_params['Kd']))) * self.dt

    def getdR(self):
        """
        Calcule la dérivée de R.
        """
        return (((self.A - self.c_params['gamma'] * self.R) + self.c_params['c0']) * self.c_params['epsilon']) * self.dt

    def getdC(self):
        """
        Calcule la dérivée de la concentration du signal chimique.
        """
        laplacian = 1 / (self.g_params['dx']**2) * np.reshape(self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0] * self.signal_size[1])), (self.signal_size[0], self.signal_size[1]))
        self.signal[0, 1:-1] -= self.fluxes[0] / self.g_params['dx']
        self.signal[-1, 1:-1] -= self.fluxes[1] / self.g_params['dx']
        self.signal[1:-1, 0] -= self.fluxes[2] / self.g_params['dx']
        self.signal[1:-1, -1] -= self.fluxes[3] / self.g_params['dx']
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        return self.dt * (self.g_params['D_sig'] * laplacian - self.c_params['aPDE'] * self.signal + self.c_params['rho'] * self.c_params['a0']) + self.dt * (self.c_params['rho'] * self.c_params['D'] * np.heaviside(self.A_grid_big, 0.5))

    def getLapC(self):
        """
        Calcule le Laplacien de la concentration du signal chimique.
        """
        return 1 / (self.g_params['dx']**2) * np.reshape(self.lap_mat.dot(np.reshape(self.signal, self.signal_size[0] * self.signal_size[1])), (self.signal_size[0], self.signal_size[1]))

    def update(self):
        """
        Met à jour l'état de la population de cellules.
        """
        self.ts += self.dt
        self.dA = self.getdA()
        if self.noise_flag:
            self.dA += self.c_params['sigma'] * np.random.normal(loc=0.0, scale=np.sqrt(self.dt), size=self.A.shape)
        self.dR = self.getdR()
        self.A_grid = accumulate_arr(self.coord, self.A, self.A_grid.shape)
        self.A_grid_big = scale(self.A_grid_big, self.A_grid, self.agent_ratio)
        self.dsignal = addcD(self.dt, self.getLapC(), self.g_params['D_sig'], self.c_params['aPDE'], self.c_params['rho'], self.c_params['a0'], self.c_params['D'], self.signal, np.heaviside(self.A_grid_big, 0.5))
        self.A += self.dA
        self.R += self.dR
        self.signal += self.dsignal
        self.signal[0, 1:-1] -= self.fluxes[0] / self.g_params['dx']
        self.signal[-1, 1:-1] -= self.fluxes[1] / self.g_params['dx']
        self.signal[1:-1, 0] -= self.fluxes[2] / self.g_params['dx']
        self.signal[1:-1, -1] -= self.fluxes[3] / self.g_params['dx']
        self.signal[0, 0] -= (self.fluxes[0] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[-1, 0] -= (self.fluxes[1] + self.fluxes[2]) / (2 * self.g_params['dx'])
        self.signal[0, -1] -= (self.fluxes[0] + self.fluxes[3]) / (2 * self.g_params['dx'])
        self.signal[-1, -1] -= (self.fluxes[1] + self.fluxes[3]) / (2 * self.g_params['dx'])
        return self.signal

### Exécution du script et production de séries temporelles de l'intensité du signal ###

def timeseries_a(a, c0,  t0, tf, freq):
    """
    Fonction pour générer des séries temporelles de l'intensité du signal.
    """
    path = 'output_files/'
    N = int(15000)  # Taille de la population
    
    # Paramètres par défaut des cellules
    cell_params_default = {
        'c0': c0, 'a': a, 'gamma': 0.5, 'Kd': 10**(-6), 'sigma': 0.15,
        'epsilon': 0.2, 'cStim': 100, 'aPDE': 10, 'rho': 1, 'D': 1000,
        'a0': 0.1, 'af': 0
    }
    
    # Paramètres par défaut de la grille
    grid_params_default = {
        'dx': 0.5, 'D_sig': 5, 'box_size_x': 50, 'box_size_y': 50,  # Ajout de 'box_size_x' et 'box_size_y'
        'agent_dim': 2 * 0.5,  # 'agent_dim' est défini indépendamment
        'num_agents': N  # Ajout de 'num_agents' pour éviter d'autres erreurs
    }
    
    noise = True

    # Initialisation de la population
    pop = dictyPop(1, 1, c_params=cell_params_default, g_params=grid_params_default, noise=noise)
    pop.A = np.zeros(pop.A.shape)
    pop.R = np.zeros(pop.R.shape)
    S = pop.update()
    
    # Initialisation des séries temporelles
    TimeSeries = np.array([(np.nan,) for i in range(int((len(S) / size)**2))])

    path = 'output_files/'
    # Création du dossier de sortie s'il n'existe pas
    os.makedirs(path, exist_ok=True)
    
    # Boucle de simulation
    for i in range(t0, tf):
        S = pop.update()
        NewCol = []
        if i % 100 == 0:
            plt.matshow(S)
            plt.title("t=" + str(i))
            # Sauvegarde dans le dossier créé
            plt.savefig(path + str(i) + '.png')
            plt.show()
        if i % save == 0:
            for i in range(0, len(S), size):
                for j in range(0, len(S), size):
                    NewCol.append(np.mean(S[i:i+size, j:j+size]) / np.mean(S))
            NCol = pd.DataFrame({'col': NewCol})
            TimeSeries = np.column_stack((TimeSeries, NCol))
    
    return TimeSeries

# Définir les variables manquantes
ll = 50  # Remplacez par la valeur appropriée
agent_dim = 2 * 0.5  # 0.5 est la valeur de 'dx'
size = 50  # Taille de la zone pour calculer les séries temporelles

# Paramètres de la simulation
t0 = 0
tf = 100000
freq = 0.1
a = 0.04

# Lancer la simulation
resultats = timeseries_a(a, 1, t0, tf, freq)

# Sauvegarder les résultats
import pandas as pd
df = pd.DataFrame(resultats)
df.to_csv("timeseries_signal.csv", index=False)

# Visualiser les résultats
import matplotlib.pyplot as plt
plt.plot(resultats)
plt.xlabel("Temps")
plt.ylabel("Intensité du signal")
plt.title("Séries temporelles de l'intensité du signal")
plt.show()